# Extração da característica de relação sinal-ruído de dados de EEG

A ideia é utilizar dados fictícios de ruído e sinal "bom"., para criarmos a relação dos dois sinais e obter como resultado um sinal de interesse "limpo".

A partir deste sinal, podemos no contexto de caracterização de foco, ainda extrair os rítmos cerebrais ou então classificar sinais com a presença ou não de foco, de forma que as amostras de sinais extraídas de um buffer sejam rotuladas com com a presença ou não de foco.

Esta atividade pode ser realizada em conjunto com um classificador comumente utilizado, como é o caso do SVM. Neste caso, uma porcetagem das amostras são utilizadas para treino e o restante para teste (p.e. 30 e 70% respectivamente).

In [1]:
# Criando dados fictícios
# pensando em um vídeo de 10 minutos, com uma taxa de amostragem de 250 Hz
#   teremos 10 * 60 * 250 = 150.000 amostras (sinal com foco)
#   teremos 1 * 60 * 250 = 15.000 amostras (basal)

import numpy as np


shape = (150000, 8)
data_focus = np.random.normal(loc=0, scale=10, size=shape).astype(np.float32)

shape = (15000, 8)
data_base = np.random.normal(loc=0, scale=10, size=shape).astype(np.float32)

In [2]:
# Estimando o ruído de fundo (utilizando o sinal basal)

# armazena uma lista com as médias de potência para cada canal
noise_power = []
for channel_data in data_base.T:
    fft_result = np.fft.fft(channel_data)
    # densidade espectral de potência (PSD)
    psd = np.abs(fft_result) ** 2
    # média da potência no intervalo de tempo sem estímulo
    base_power = np.mean(psd)
    noise_power.append(base_power)
# média das médias de potência de todos os canais para estimar o ruído de fundo
estimated_background_noise = np.mean(noise_power)
print(estimated_background_noise)

1497838.9


Vamos calcular o SNR de "banda estreita". Pode ser observado pela seguinte equação:

$SNR_{banda\ estreita} = 10 \cdot \log_{10}\left(\frac{\text{energia total do espectro}}{\text{média das amplitudes nas frequências vizinhas}}\right)$

Já o SNR de banda larga é definido da seguinte forma:

$SNR_{banda\ larga} = 10 \cdot \log_{10}\left(\frac{\text{energia total do espectro}}{\text{energia total do espectro de amplitude}}\right)$

In [3]:
# agora vamos adaptar ambas características 
# aplicando para o nosso sinal de interesse

# forçando (estragando) valor de "estimated_background_noise" para não sobrar valores negativos
target_amplitudes_adjusted = data_focus - estimated_background_noise

# subtraindo o ruído de fundo das amplitudes
narrow_band_SNR = 10 * np.log10(target_amplitudes_adjusted / estimated_background_noise)
print(narrow_band_SNR)
print(narrow_band_SNR.shape)

total_power = np.sum(target_amplitudes_adjusted)
wide_band_SNR = 10 * np.log10(target_amplitudes_adjusted / total_power)
print(wide_band_SNR)
print(wide_band_SNR.shape)


[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
(150000, 8)
[[-60.79186  -60.791832 -60.791874 ... -60.791878 -60.791782 -60.791798]
 [-60.79178  -60.79184  -60.791862 ... -60.791824 -60.791813 -60.791798]
 [-60.791786 -60.79177  -60.791775 ... -60.791813 -60.79184  -60.791733]
 ...
 [-60.7918   -60.791794 -60.791824 ... -60.7918   -60.79185  -60.791847]
 [-60.791855 -60.791855 -60.791813 ... -60.791836 -60.7918   -60.79178 ]
 [-60.79185  -60.79184  -60.79182  ... -60.79182  -60.791824 -60.79182 ]]
(150000, 8)


/tmp/ipykernel_9636/2533300642.py:8: RuntimeWarning: invalid value encountered in log10
  narrow_band_SNR = 10 * np.log10(target_amplitudes_adjusted / estimated_background_noise)


### Tarefa para aplicação das características SNR:

Agora que temos os dois vetores de características SNR, podemos utilizar buffers com e sem a evocação dos rítmos que caracterizam o foco.

#### Divisão dos dados

Utilizando a iteração (por exemplo, de 5 segundos caracterizada pela janela) realizada no sinal a cada ~1 segundo, realize a rotulação dos dados de interesse (Beta e Gamma). Ou seja, cada amostra sera um sinal de 5 segundos (1250 pontos de 8 canais). A janela que não for qualificada como Beta ou Gama por exemplo, poderá ser rotulada com "desfoque". Se acharem interessante, adicionar rótulos do ritmo Theta também.

No caso do sinal que representa o basal (se tiverem) poderá pegar um único sinal de aproximadamente 30 segundos para ser utilizado na equação de ruído, que irá ter como resultado um único valor. Lembrando que o valor de ruído deve atuar no sinal no domínio da frequência.

#### Classificação

Em nossos dados simulados, temos 150.000 pontos com 8 canais. A utilização desses dados funcionará da seguinte forma para a criação do vetor de características:

- 150.000 (pontos totais) / 250 (taxa de amostragem) = 600 segundos
- 600 / 5 (tamanho da janela sem sobreposição) = 120 amostras

| 1   | SNRw1                | SNRw2 | ... | SNRw8 | SNRn1 | SNRn2 | ... | SNRn8 |
|-----|----------------------|-------|-----|-------|-------|-------|-----|-------|
| 2   | [w1, w2, ..., w1250] |       |     |       |       |       |     |       |
| 3   |                      |       |     |       |       |       |     |       |
| ... |                      |       |     |       |       |       |     |       |
| 120 |                      |       |     |       |       |       |     |       |

- Agora transforme cada um dos vetores de pontos no domínio da frequência (1250 pontos) em um único valor real. Neste caso pode ser utilizado tanto a média como a mediana (ou ambos). Se utilizarmos as duas, teremos no final 32 colunas de características:
    - 8 canais
    - SNR narrow e SNR wide (2)
    - Média e mediana (2)

| 1   | 1   | ... | 32 |
|-----|-----|-----|----|
| 2   | w'  | ... |    |
| 3   | ... |     |    |
| ... |     |     |    |
| 120 |     |     |    |



Após obter o vetor de característica, realizar a divisão dos dados em treinamento e teste (normalmente uma proporção de 70 e 30% respectivamente) e aplicar para o classificador SVM.

**PLUS**: Ao final da tarefa, verificar a melhora dos resultados utilizando um seletor de características. Neste caso, podemos utilizar o RFE (*Recursive Feature Elimination*) em uma fase anterior a classificação para reduzir as 32 características se for necessário.

